In [1]:
from ssd import build_ssd

## box_utils裡面很多Function，可以看看是怎麼設計的
from layers.box_utils import * 
from torch.autograd import Variable
from layers import functions
from layers import modules
from math import sqrt 
from itertools import product
from torch.autograd import Function

import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import argparse
import pickle
import torch.nn.functional as F


In [2]:
## 詳細模型結構可以參考ssd.py
ssd_net=build_ssd('train', size=300, num_classes=21)

C:\Users\andy\Desktop\course\deep_learning_marathon\D29 - D31：程式導讀、實作\Object Detection 程式導讀\ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)


## 默認Config檔案在data/config.py內

In [3]:
ssd_net.cfg

{'num_classes': 21,
 'lr_steps': (80000, 100000, 120000),
 'max_iter': 120000,
 'feature_maps': [38, 19, 10, 5, 3, 1],
 'min_dim': 300,
 'steps': [8, 16, 32, 64, 100, 300],
 'min_sizes': [30, 60, 111, 162, 213, 264],
 'max_sizes': [60, 111, 162, 213, 264, 315],
 'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
 'variance': [0.1, 0.2],
 'clip': True,
 'name': 'VOC'}

In [4]:
cfg = {
    'num_classes': 21,
    'lr_steps': (80000, 100000, 120000),
    'max_iter': 120000,
    'feature_maps': [38, 19, 10, 5, 3, 1],
    'min_dim': 300,
    'steps': [8, 16, 32, 64, 100, 300],
    'min_sizes': [30, 60, 111, 162, 213, 264],
    'max_sizes': [60, 111, 162, 213, 264, 315],
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
    'variance': [0.1, 0.2],
    'clip': True,
    'name': 'VOC',
}

### 'aspect_ratios' : 使用六張Feature Map，每一張上方有預設的anchor boxes，Boxes aspect ratio可以自己設定
### 'feature_maps' : 使用feature map大小為[38x38, 19x19, 10x10, 5x5, 3x3, 1x1]
### 'min_sizes'、'max_sizes'可藉由下方算式算出，由作者自行設計
### 'steps' : Feature map回放回原本300*300的比例，如38要回放為300大概就是8倍
### 'variance' : Training 的一個trick，加速收斂，詳見：https://github.com/rykov8/ssd_keras/issues/53

---

## 'min_sizes'、'max_sizes' 計算

In [5]:
import math
## source:https://blog.csdn.net/gbyy42299/article/details/81235891
min_dim = 300   ## 维度
# conv4_3 ==> 38 x 38
# fc7 ==> 19 x 19
# conv6_2 ==> 10 x 10
# conv7_2 ==> 5 x 5
# conv8_2 ==> 3 x 3
# conv9_2 ==> 1 x 1
mbox_source_layers = ['conv4_3', 'fc7', 'conv6_2', 'conv7_2', 'conv8_2', 'conv9_2'] ## prior_box來源層，可以更改。很多改進都是基於此處的調整。
# in percent %
min_ratio = 20 ## 這裡即是論文中所說的Smin的= 0.2，Smax的= 0.9的初始值，經過下面的運算即可得到min_sizes，max_sizes。
max_ratio = 90
step = int(math.floor((max_ratio - min_ratio) / (len(mbox_source_layers) - 2)))## 取一個間距步長，即在下面用於循環給比取值時起一個間距作用。可以用一個具體的數值代替，這裡等於17。
min_sizes = []  ## 經過以下運算得到min_sizes和max_sizes。
max_sizes = []
for ratio in range(min_ratio, max_ratio + 1, step):
    ## 從min_ratio至max_ratio + 1每隔步驟= 17取一個值賦值給比。注意範圍函數的作用。
    ## min_sizes.append（）函數即把括號內部每次得到的值依次給了min_sizes。
    min_sizes.append(min_dim * ratio / 100.)
    max_sizes.append(min_dim * (ratio + step) / 100.)
min_sizes = [min_dim * 10 / 100.] + min_sizes
max_sizes = [min_dim * 20 / 100.] + max_sizes

## steps: 這一步要仔細理解，即計算卷積層產生的prior_box距離原圖的步長，先驗框中心點的坐標會乘以step，
## 相當於從特徵映射位置映射回原圖位置，比如conv4_3輸出特徵圖大小為38 *38，而輸入的圖片為300* 300，
## 所以38 *8約等於300，所以映射步長為8.這是針對300* 300的訓練圖片。
steps = [8, 16, 32, 64, 100, 300]  
aspect_ratios = [[2], [2, 3], [2, 3], [2, 3], [2], [2]]
 
print('min_sizes: ',min_sizes)
print('max_sizes: ',max_sizes)


min_sizes:  [30.0, 60.0, 111.0, 162.0, 213.0, 264.0]
max_sizes:  [60.0, 111.0, 162.0, 213.0, 264.0, 315.0]


---

## Default anchor boxes設計原理，看懂收穫很多
##### 可以理解 SSD原文中 8732個anchors是怎麼來的
##### 38×38×4+19×19×6+10×10×6+5×5×6+3×3×4+1×1×4=8732

In [6]:
class PriorBox(object):
    """Compute priorbox coordinates in center-offset form for each source
    feature map.
    """
    def __init__(self, cfg):
        super(PriorBox, self).__init__()
        self.image_size = cfg['min_dim']
        # number of priors for feature map location (either 4 or 6)
        self.num_priors = len(cfg['aspect_ratios'])
        self.variance = cfg['variance'] or [0.1]
        self.feature_maps = cfg['feature_maps']
        self.min_sizes = cfg['min_sizes']
        self.max_sizes = cfg['max_sizes']
        self.steps = cfg['steps']
        self.aspect_ratios = cfg['aspect_ratios']
        self.clip = cfg['clip']
        self.version = cfg['name']
        for v in self.variance:
            if v <= 0:
                raise ValueError('Variances must be greater than 0')

    def forward(self):
        mean = []
        '''依照Feature map大小找出所有的pixel 中心'''
        '''下方這兩個loop會找出W個x軸pixel對上W個y軸pixel，假如現在是在38x38的feature map上，就會有38x38個值'''
        '''ex. [0,1],[0,2]..[0,37] [1,1],[1,2]..[1,37]..........[37,37]'''
        for k, f in enumerate(self.feature_maps):
            for i, j in product(range(f), repeat=2):
                f_k = self.image_size / self.steps[k] ## 如self.steps==8，就是先將原圖size normalize(/300)後再乘上8
                # unit center x,y
                '''中心點'''
                cx = (j + 0.5) / f_k
                cy = (i + 0.5) / f_k

                # aspect_ratio: 1
                # rel size: min_size
                '''/self.image_size 就是在做normalization '''
                s_k = self.min_sizes[k]/self.image_size
                '''小的正方形box'''
                mean += [cx, cy, s_k, s_k]

                # aspect_ratio: 1
                # rel size: sqrt(s_k * s_(k+1))
                '''大的正方形box'''
                s_k_prime = sqrt(s_k * (self.max_sizes[k]/self.image_size))
                mean += [cx, cy, s_k_prime, s_k_prime]

                # rest of aspect ratios
                for ar in self.aspect_ratios[k]:
                    '''aspect ratio 2,3'''
                    mean += [cx, cy, s_k*sqrt(ar), s_k/sqrt(ar)]
                    '''aspect ratio 1/2,1/3'''
                    mean += [cx, cy, s_k/sqrt(ar), s_k*sqrt(ar)]
        # back to torch land
        output = torch.Tensor(mean).view(-1, 4)
        if self.clip:
            output.clamp_(max=1, min=0)
        return output

In [7]:
PriorBox_Demo=PriorBox(cfg)

In [8]:
print(PriorBox_Demo.forward().shape)

torch.Size([8732, 4])


---

## Loss 如何設計

In [9]:
class MultiBoxLoss(nn.Module):

    def __init__(self, num_classes, overlap_thresh, prior_for_matching,
                 bkg_label, neg_mining, neg_pos, neg_overlap, encode_target,
                 use_gpu=True):
        super(MultiBoxLoss, self).__init__()
        self.use_gpu = use_gpu
        '''有幾類'''
        self.num_classes = num_classes
        '''判定為正樣本的threshold，一般設為0.5'''
        self.threshold = overlap_thresh
        '''background自己會有一類，不用Label，假如我們有20類一樣標註0-19，下方會自己空出一類給background'''
        self.background_label = bkg_label
        self.encode_target = encode_target
        self.use_prior_for_matching = prior_for_matching
        '''OHEM，找出分得最不好的樣品，也就是confidence score比較低的正負樣品'''
        self.do_neg_mining = neg_mining
        '''負樣品與正樣品的比例，通常是3:1'''
        self.negpos_ratio = neg_pos
        self.neg_overlap = neg_overlap
        self.variance = cfg['variance']
     

    def forward(self, predictions, targets):

        '''prediction會output三個值'''
        '''loc shape: bounding box 資訊，torch.size(batch_size,num_priors,4)'''
        '''conf shape: 每一個bounding box 的信心程度，torch.size(batch_size,num_priors,num_classes)'''
        '''priors shape: 預設的defaul box， torch.size(num_priors,4)'''
        loc_data, conf_data, priors = predictions
        num = loc_data.size(0)
        priors = priors[:loc_data.size(1), :]
        num_priors = (priors.size(0))
        num_classes = self.num_classes

        # match priors (default boxes) and ground truth boxes
        loc_t = torch.Tensor(num, num_priors, 4)
        conf_t = torch.LongTensor(num, num_priors)
        for idx in range(num):
            truths = targets[idx][:, :-1].data
            labels = targets[idx][:, -1].data
            defaults = priors.data
            '''jaccard 計算每一個BBOX與ground truth的IOU'''
            match(self.threshold, truths, defaults, self.variance, labels,
                  loc_t, conf_t, idx)
        if self.use_gpu:
            loc_t = loc_t.cuda()
            conf_t = conf_t.cuda()
        '''用Variable包裝'''
        loc_t = Variable(loc_t, requires_grad=False)
        conf_t = Variable(conf_t, requires_grad=False)

        pos = conf_t > 0
        num_pos = pos.sum(dim=1, keepdim=True)


        pos_idx = pos.unsqueeze(pos.dim()).expand_as(loc_data)
        loc_p = loc_data[pos_idx].view(-1, 4)
        loc_t = loc_t[pos_idx].view(-1, 4)
        '''smooth_l1_loss 計算bounding box regression'''
        loss_l = F.smooth_l1_loss(loc_p, loc_t, size_average=False)

        # Compute max conf across batch for hard negative mining
        batch_conf = conf_data.view(-1, self.num_classes)
        loss_c = log_sum_exp(batch_conf) - batch_conf.gather(1, conf_t.view(-1, 1))

        # Hard Negative Mining
        loss_c = loss_c.view(num, -1)
        loss_c[pos] = 0
        '''排列confidence 的分數'''
        _, loss_idx = loss_c.sort(1, descending=True)
        _, idx_rank = loss_idx.sort(1)
        num_pos = pos.long().sum(1, keepdim=True)
        '''負樣品取出數量 == negpos_ratio*num_pos'''
        num_neg = torch.clamp(self.negpos_ratio*num_pos, max=pos.size(1)-1)
        neg = idx_rank < num_neg.expand_as(idx_rank)

        # Confidence Loss Including Positive and Negative Examples
        pos_idx = pos.unsqueeze(2).expand_as(conf_data)
        neg_idx = neg.unsqueeze(2).expand_as(conf_data)
        conf_p = conf_data[(pos_idx+neg_idx).gt(0)].view(-1, self.num_classes)
        targets_weighted = conf_t[(pos+neg).gt(0)]
        '''用cross_entropy做分類'''
        loss_c = F.cross_entropy(conf_p, targets_weighted, size_average=False)

        # Sum of losses: L(x,c,l,g) = (Lconf(x, c) + αLloc(x,l,g)) / N
        #double轉成torch.float64
        N = num_pos.data.sum().double()
        loss_l = loss_l.double()
        loss_c = loss_c.double()
        loss_l /= N
        loss_c /= N
        return loss_l, loss_c


## 產生我們Loss function，注意這裡的class要包含背景

In [10]:
Use_cuda=False
criterion = MultiBoxLoss(21, 0.5, True, 0, False, 3, 0.5,False, Use_cuda,)

----

## 基本設定

In [11]:
ssd_net=build_ssd('train', size=300, num_classes=21)
use_pretrained=False
if use_pretrained:
    ssd_net.load_weights('./demo/ssd300_mAP_77.43_v2.pth')
net=ssd_net

In [12]:
'''要不要使用gpu'''
Use_cuda=False

'''tensor type會依照cpu或gpu有所不同'''
if torch.cuda.is_available():
    if Use_cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
    else:
        print("WARNING: It looks like you have a CUDA device, but aren't " +
              "using CUDA.\nRun with --cuda for optimal training speed.")
        torch.set_default_tensor_type('torch.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')


'''使用GPU時模型要轉成cuda'''
if Use_cuda:
    net = net.cuda()
    
batch_size_=32
optimizer = optim.Adam(net.parameters(),lr=0.00001/batch_size_)

---

## 訓練

## 這裡我們先示範輸入的 image,Label格式，真正在訓練時，準備成一樣格式即可

In [13]:
'''輸入影像格式，假設batch size 為 4'''
image_in=torch.tensor(torch.rand(4,3,300,300),dtype=torch.float32)
'''Label格式，沒有固定長度，看圖像中有幾個label就有幾個'''
label_0=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 19.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]
label_1=[[ 0.1804,  0.6076,  0.7701,  0.8485, 13.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 11.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 7.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 5.0000],]
label_2=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 14.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]
label_3=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 19.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]

C:\Users\andy\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [14]:
epochs=300
iteration=1000

In [15]:
for epoch in range(epochs):
    n=0
    loss_sum=[]
    loc_loss=[]
    conf_loss=[]
    for number__ in range(iteration) :
        '''要用Variable包裝tensor才能送入模型'''
        if Use_cuda:
            image_ = Variable(image_in.cuda())
            y = [Variable(torch.tensor(label_0).cuda(), volatile=True),Variable(torch.tensor(label_1).cuda(), 
                volatile=True),Variable(torch.tensor(label_2).cuda(), volatile=True),Variable(torch.tensor(label_3).cuda(), volatile=True)]      
        else:
            image_ = Variable(image_in)
            y = [Variable(torch.tensor(label_0), volatile=True),Variable(torch.tensor(label_1), 
                volatile=True),Variable(torch.tensor(label_2), volatile=True),Variable(torch.tensor(label_3), volatile=True)]

        '''Forward Pass'''
        out = net(image_)
        '''Regression Loss and Classification Loss'''
        loss_l,loss_c = criterion(out,y )
        loss = loss_l+ loss_c
        '''Backward'''
        loss.backward()

        loc_loss.append(loss_l.data.cpu().numpy())
        conf_loss.append(loss_c.data.cpu().numpy())
        loss_sum.append(loss.data.cpu().numpy())
        '''更新參數'''
        optimizer.step()
        '''清空Gradients'''
        optimizer.zero_grad()
        
        n+=1
        if n%10==0:
            print('BBOX Regression Loss: ', np.mean(loc_loss))
            print('Classification Loss: ', np.mean(conf_loss))
    '''儲存權重'''
    torch.save(ssd_net.state_dict(),'./Ｗeights.pth')

C:\Users\andy\anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
C:\Users\andy\anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  from ipykernel import kernelapp as app
C:\Users\andy\anaconda3\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


BBOX Regression Loss:  2.515607339364511
Classification Loss:  13.751190637659144
BBOX Regression Loss:  2.515473033763744
Classification Loss:  13.667516468189381
BBOX Regression Loss:  2.5153388411910442
Classification Loss:  13.596361325110921
BBOX Regression Loss:  2.515204514397515
Classification Loss:  13.52810999552409
BBOX Regression Loss:  2.515070116961444
Classification Loss:  13.461738597728585
BBOX Regression Loss:  2.514935764265649
Classification Loss:  13.397173883885513
BBOX Regression Loss:  2.514801320070943
Classification Loss:  13.335163935908565
BBOX Regression Loss:  2.5146667763038915
Classification Loss:  13.276106940375433
BBOX Regression Loss:  2.5145321253395863
Classification Loss:  13.219418724279834
BBOX Regression Loss:  2.5143972947862414
Classification Loss:  13.164638830114296
BBOX Regression Loss:  2.51426228995275
Classification Loss:  13.111524125943683
BBOX Regression Loss:  2.514127076702353
Classification Loss:  13.059693164589964
BBOX Regressio

BBOX Regression Loss:  2.4835178233959057
Classification Loss:  12.150732647931132
BBOX Regression Loss:  2.4832094545717593
Classification Loss:  12.150236567744502
BBOX Regression Loss:  2.4828936447331937
Classification Loss:  12.149732650945216
BBOX Regression Loss:  2.4825709802133065
Classification Loss:  12.14922728361907
BBOX Regression Loss:  2.4822417308666087
Classification Loss:  12.148719640661168
BBOX Regression Loss:  2.4819053367332176
Classification Loss:  12.148208505135996
BBOX Regression Loss:  2.4815621148972284
Classification Loss:  12.147692725771948
BBOX Regression Loss:  2.481212011973063
Classification Loss:  12.147170935736762
BBOX Regression Loss:  2.480854260479962
Classification Loss:  12.14664218729906
BBOX Regression Loss:  2.4804879534686055
Classification Loss:  12.14610565185547
BBOX Regression Loss:  2.480112280026831
Classification Loss:  12.145559980090617
BBOX Regression Loss:  2.479726424040618
Classification Loss:  12.14500461154514
BBOX Regress

BBOX Regression Loss:  1.9131357970061127
Classification Loss:  10.687947252061631
BBOX Regression Loss:  1.910551890620479
Classification Loss:  10.674821415653934
BBOX Regression Loss:  1.9079368968068815
Classification Loss:  10.661669695818865
BBOX Regression Loss:  1.9053091261121964
Classification Loss:  10.648452645760994
BBOX Regression Loss:  1.9026689374005354
Classification Loss:  10.635165540907117
BBOX Regression Loss:  1.9000094401983567
Classification Loss:  10.62180251133295
BBOX Regression Loss:  1.8973233883973784
Classification Loss:  10.608343570446843
BBOX Regression Loss:  1.8946142178994638
Classification Loss:  10.594778258712203
BBOX Regression Loss:  1.8918949252784008
Classification Loss:  10.581094862698528
BBOX Regression Loss:  1.889168138857241
Classification Loss:  10.567276588722509
BBOX Regression Loss:  1.886433514681729
Classification Loss:  10.553306738856666
BBOX Regression Loss:  1.8836915875658575
Classification Loss:  10.539169132562332
BBOX Reg

BBOX Regression Loss:  0.46766161035608367
Classification Loss:  4.018459517867477
BBOX Regression Loss:  0.4609999179840088
Classification Loss:  3.9996802294695817
BBOX Regression Loss:  0.45438610948162306
Classification Loss:  3.9808708661868244
BBOX Regression Loss:  0.44771619196291323
Classification Loss:  3.9622446837248626
BBOX Regression Loss:  0.4410200585259332
Classification Loss:  3.9438146294487852
BBOX Regression Loss:  0.43437543857244787
Classification Loss:  3.925484240496601
BBOX Regression Loss:  0.4277373364362768
Classification Loss:  3.9072672727877507
BBOX Regression Loss:  0.4211260349662216
Classification Loss:  3.889149983723958
BBOX Regression Loss:  0.4145298369136858
Classification Loss:  3.871193478431231
BBOX Regression Loss:  0.4079753434216534
Classification Loss:  3.853375074598525
BBOX Regression Loss:  0.4014995009810836
Classification Loss:  3.8356775405832417
BBOX Regression Loss:  0.39511754748262007
Classification Loss:  3.8180684525289648
BBOX

...

BBOX Regression Loss:  0.00012852411749133584
Classification Loss:  0.0003550365104589587
BBOX Regression Loss:  0.0001283851681799485
Classification Loss:  0.00035353945913138216
BBOX Regression Loss:  0.0001282482898560326
Classification Loss:  0.00035205570331280246
BBOX Regression Loss:  0.00012811425902244
Classification Loss:  0.00035058513658495997
BBOX Regression Loss:  0.00012798208677977847
Classification Loss:  0.000349127887212626
BBOX Regression Loss:  0.00012785202002370375
Classification Loss:  0.00034768373288872236
BBOX Regression Loss:  0.00012772371412548062
Classification Loss:  0.0003462523662623156
BBOX Regression Loss:  0.0001275981874007466
Classification Loss:  0.0003448337196206713
BBOX Regression Loss:  0.00012747899192258898
Classification Loss:  0.00034342714346373676
BBOX Regression Loss:  0.0004267833594908227
Classification Loss:  0.00034205776468419904
BBOX Regression Loss:  0.0014763166432414734
Classification Loss:  0.0003425459815690965
BBOX Regressi

BBOX Regression Loss:  0.002346501204329795
Classification Loss:  0.0003127674623909924
BBOX Regression Loss:  0.0022640210841124464
Classification Loss:  0.0003137933957775025
BBOX Regression Loss:  0.002187379028324146
Classification Loss:  0.0003146029801832305
BBOX Regression Loss:  0.0021159778289332755
Classification Loss:  0.0003152232162778101
BBOX Regression Loss:  0.002049299495835087
Classification Loss:  0.0003156773926529252
BBOX Regression Loss:  0.0019868922588975125
Classification Loss:  0.000315986030257731
BBOX Regression Loss:  0.001928359499638814
Classification Loss:  0.00031616654348369966
BBOX Regression Loss:  0.001873350159477288
Classification Loss:  0.0003162332636949122
BBOX Regression Loss:  0.0018215568115821108
Classification Loss:  0.0003161990732027619
BBOX Regression Loss:  0.0017727051799496017
Classification Loss:  0.00031607531572854707
BBOX Regression Loss:  0.001726550529778019
Classification Loss:  0.0003158711045482215
BBOX Regression Loss:  0.0

BBOX Regression Loss:  0.001835463492778502
Classification Loss:  0.00022418022312898448
BBOX Regression Loss:  0.0024787430955572724
Classification Loss:  0.00022696356014956744
BBOX Regression Loss:  0.002585855634210424
Classification Loss:  0.00023109792041833756
BBOX Regression Loss:  0.0025412529517241096
Classification Loss:  0.00023547716658871222
BBOX Regression Loss:  0.0024572547162202522
Classification Loss:  0.0002395238192683017
BBOX Regression Loss:  0.0023647989450103573
Classification Loss:  0.00024306970496996397
BBOX Regression Loss:  0.002274717472962184
Classification Loss:  0.00024613990321939374
BBOX Regression Loss:  0.00218983823781902
Classification Loss:  0.00024877550494339733
BBOX Regression Loss:  0.0021106178987375926
Classification Loss:  0.0002510300171468672
BBOX Regression Loss:  0.002036884093857709
Classification Loss:  0.0002529481171652041
BBOX Regression Loss:  0.001968208242764588
Classification Loss:  0.0002545794194919013
BBOX Regression Loss:

BBOX Regression Loss:  0.0001300043653248361
Classification Loss:  0.0003855948553731043
BBOX Regression Loss:  0.00012867850093139645
Classification Loss:  0.00038328786703572924
BBOX Regression Loss:  0.0001274260366766679
Classification Loss:  0.0003810180972019831
BBOX Regression Loss:  0.00012623782341296855
Classification Loss:  0.0003787843847482802
BBOX Regression Loss:  0.00012510831334761212
Classification Loss:  0.00037658719232591686
BBOX Regression Loss:  0.0001240305855502317
Classification Loss:  0.0003744255130489667
BBOX Regression Loss:  0.00012300175860420696
Classification Loss:  0.0003722993771252395
BBOX Regression Loss:  0.0001220198704258484
Classification Loss:  0.00037020921183664814
BBOX Regression Loss:  0.00012108211941181571
Classification Loss:  0.0003681522060124349
BBOX Regression Loss:  0.00012019039304168135
Classification Loss:  0.00036612730418816646
BBOX Regression Loss:  0.00011933932762110123
Classification Loss:  0.0003641337911701865
BBOX Regre

KeyboardInterrupt: 

## 想要Train VOC,COCO可以參考原Github:https://github.com/amdegroot/ssd.pytorch